In [1]:
import geopandas as gpd
import xarray as xr
import numpy as np
import rioxarray as rxr
import os
from math import e
import matplotlib.pyplot as plt

In [2]:
IDir = r'/Volumes/PhD/imagery/masters/output/dNBR/basin_12042_dNBR.tif'
ODir = r'/Volumes/PhD/imagery/masters/output/ROC/'

In [3]:
def M1(dnbr, 
          save=False):
    
    # Coefficients for Southern California
    b = -3.63
    b_1 = 0.41
    b_2 = 0.67
    b_3 = 0.7
    
    # File naming
    

    dnbr_flat = dnbr.values.flatten()
    pixarr = np.empty((dnbr_flat.shape[0]), dtype= float)
    pixarr[:] = np.nan
    x = np.where(np.isnan(dnbr_flat) == False)
        
    # Getting the data from dnbr_flat
    dnbrdata = dnbr_flat[x]
    dnbrdata = np.mean(dnbrdata)
        
    # Pulling tif metadata
    metadata = dnbr.attrs
    x_1 = float(metadata.get('X1'))
    x_3 = float(metadata.get('X3'))
    
    R = 19.56

    lnx =  b + (b_1 * x_1 * R) + (b_2 * dnbrdata * R) + (b_3 * x_3 * R)
    prob = (e ** lnx) / (1.0 + e ** lnx)
    return prob
    

In [4]:
q = rxr.open_rasterio(IDir)
dnbr_flat = q.values.flatten()
pixarr = np.empty((dnbr_flat.shape[0]), dtype= float)
pixarr[:] = np.nan
x = np.where(np.isnan(dnbr_flat) == False)
        
# Getting the data from dnbr_flat
dnbrdata = dnbr_flat[x]
dnbrdata = np.mean(dnbrdata)

print(dnbrdata)
y = M1(q)
print(y)

0.21659964
0.9291953


In [6]:
shp = r'/Volumes/PhD/imagery/masters/baer/thm2017_Segment_DFPredictions_15min_40mmh.shp'
shp = gpd.read_file(shp)
basin_id = 12042
basin_data = shp[shp['Segment_ID'] == basin_id]
print(basin_data['P'])

12424    0.414864
Name: P, dtype: float64
